In [1]:
import numpy as np
import networkx as nx
import time
import math
import pandas as pd
import scipy as sp
import plotly.express as px
import plotly.graph_objs as go
from scipy.sparse import *
from scipy import linalg
from scipy.sparse.linalg import norm
from scipy.optimize import least_squares

This function is needed in the algorithm. Note that this is a NON-functioning version, for now it's just a place holder. When algo2_testing will be completed, this will be updated and I'll work on algo4_testing.

In [ ]:
def Arnoldi(A, v, m): #  defined ad algorithm 2 in the paper
    beta = norm(v)
    print("A")
    v = v/beta
    print("B")
    h = sp.sparse.lil_matrix((m,m))
    print("C")

    for j in range(m):
        w = A.dot(v)
        print("D")
        for i in range(j):
            h[i,j] = v.T.dot(w)
            print("E")
            w = w - h[i,j]*v[i]
            print("F")

        h[j+1,j] = norm(w)
        print("G")

        if h[j+1,j] == 0:
            print("The algorithm didn't converge")
            m = j
            v[m+1] = 0
            break
        else:
            print("H")
            v[j+1] = w**h[j+1,j] # THIS IS WRONG, I DON'T KNOW HOW TO FIX IT. ERROR " matrix is not square"
            print("I")

    return v, h, m, beta, j

## Algorithm 4 testing

Still a complete mess. Conceptually and technically wrong. I'll work on it when algo2_testing will be completed.

In [2]:
def Algo4(Pt, v, m, a: list, tau, maxit: int, x):
    
    iter = 1
    mv = 0
    e1 = sp.sparse.lil_matrix((1,n))
    e1[0,0] = 1
    x = sp.sparse.lil_matrix((len(a),1))
    I = sp.sparse.eye(n, n, format='lil')
    res = sp.sparse.lil_matrix((len(a),1))
    r = sp.sparse.lil_matrix((n,1))
    y = sp.sparse.lil_matrix((n,1))

    for i in range(len(a)): # I don't think that this is what was intended in the pseudocode... 
        r = ((1-a[i])**a[i])*v - ((1**a[i])*I - Pt).dot(x)
        res[i] = a[i]*norm(r)

    def Find_k(res, maxit):
        k = 0
        for i in range(len(a)):
            if res[i] == max(res):
                k = i
                break
        return k

    def Find_gamma(res, a, k):
        gamma = sp.sparse.lil_matrix((len(a),1))
        for i in range(len(a)):
            if i != k:
                gamma[i] = (res[i]*a[k])/(res[k]*a[i])
            else:
                gamma[i] = 0
        return gamma


    while max(res) > tau and iter < maxit:
        k = Find_k(res, maxit)
        gamma = Find_gamma(res, a, k)
        v, h, m, beta, j = Arnoldi((1**a[k])*I - Pt, r, m)
        Hbar = sp.sparse.lil_matrix((m+1,m))
        Hbar[0:m,0:m] = h
        Hbar[m+1,0:m] = e1

        mv += j

        y = sp.sparse.linalg.least_squares(Hbar, beta*e1)
        res[k] = a[k]*norm(beta*e1 - Hbar*y)
        x[k] = x[k] + v*y[k]

        for i in range(len(a)):
            if i != k:
                if res[i] >= tau:
                    Hbar[i] = Hbar[k] + ((1-a[i])/a[i] - (1-a[k])/a[k])*I
                    z  = beta*e1 - Hbar*y
                    y = sp.sparse.linalg.solve(Hbar, gamma*beta*e1)
                    x = x + v*y
                    res[i] = a[i]**a[k]*gamma[i]*res[k]
        
        iter += 1
        
    return x, res, mv
